# Explorador de Hallazgos
Herramienta interactiva para revisar resultados del pipeline de red-teaming.

In [ ]:
# Celda 1: setup y carga de librerías
import json, glob
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, Markdown
import plotly.graph_objects as go
import plotly.express as px

# DataFrames globales
df = pd.DataFrame()
df_filtered = pd.DataFrame()

In [ ]:
# Celda 2: selector de archivo
file_dropdown = widgets.Dropdown(options=glob.glob('data/findings/*.json'), description='Archivo')
load_button = widgets.Button(description='Cargar')
selector_box = widgets.HBox([file_dropdown, load_button])
display(selector_box)

# Función para cargar el archivo seleccionado
def load_data(_):
    global df, df_filtered
    if file_dropdown.value:
        with open(file_dropdown.value, 'r', encoding='utf-8') as f:
            data = json.load(f)
        df = pd.DataFrame(data)
        for col in ['id','category','flags','severity','reproducibility','prompt','response','regenerated']:
            if col not in df.columns:
                df[col] = ''
        if 'notes' not in df.columns:
            df['notes'] = ''
        df_filtered = df.copy()
        try:
            update_filters()
            update_hist()
            update_table()
        except NameError:
            pass

load_button.on_click(load_data)

In [ ]:
# Celda 3: visualización tabular y comparador
table_output = widgets.Output()
compare_output = widgets.Output()
row_selector = widgets.Dropdown(description='ID')
notes_area = widgets.Textarea(description='Notas', layout=widgets.Layout(width='100%', height='100px'))
notes_preview = widgets.Output()

# Actualiza la tabla con el DataFrame filtrado
def update_table():
    table_output.clear_output()
    with table_output:
        if not df_filtered.empty:
            fig = go.FigureWidget(data=[go.Table(
                header=dict(values=list(df_filtered.columns)),
                cells=dict(values=[df_filtered[c] for c in df_filtered.columns])
            )])
            fig.update_layout(height=300)
            display(fig)
    row_selector.options = df_filtered['id'].tolist()

# Muestra comparación lado a lado
def show_row(change):
    compare_output.clear_output()
    rid = change['new']
    if rid:
        row = df_filtered[df_filtered['id']==rid].iloc[0]
        with compare_output:
            display(widgets.HBox([
                widgets.HTML(f"<b>Original</b><br><pre>{row['response']}</pre>", layout=widgets.Layout(width='50%')),
                widgets.HTML(f"<b>Re-generada</b><br><pre>{row.get('regenerated','')}</pre>", layout=widgets.Layout(width='50%'))
            ]))
        notes_area.value = row.get('notes','')
        notes_preview.clear_output()
        with notes_preview:
            display(Markdown(row.get('notes','')))

row_selector.observe(show_row, names='value')

# Guarda notas y renderiza markdown
def save_notes(change):
    rid = row_selector.value
    if rid in df['id'].values:
        df.loc[df['id']==rid,'notes'] = change['new']
        df_filtered.loc[df_filtered['id']==rid,'notes'] = change['new']
        notes_preview.clear_output()
        with notes_preview:
            display(Markdown(change['new']))

notes_area.observe(save_notes, names='value')

display(table_output, row_selector, compare_output, notes_area, notes_preview)

In [ ]:
# Celda 4: filtros interactivos
category_filter = widgets.Dropdown(description='Categoría')
severity_filter = widgets.Dropdown(description='Riesgo')

# Actualiza opciones de filtros según los datos
def update_filters():
    category_filter.options = ['Todos'] + sorted([c for c in df['category'].dropna().unique()])
    severity_filter.options = ['Todos'] + sorted([c for c in df['severity'].dropna().unique()])

# Aplica filtros y refresca vistas
def apply_filters(change=None):
    global df_filtered
    df_filtered = df.copy()
    if category_filter.value and category_filter.value != 'Todos':
        df_filtered = df_filtered[df_filtered['category']==category_filter.value]
    if severity_filter.value and severity_filter.value != 'Todos':
        df_filtered = df_filtered[df_filtered['severity']==severity_filter.value]
    update_table()
    update_hist()

category_filter.observe(apply_filters, names='value')
severity_filter.observe(apply_filters, names='value')

display(widgets.HBox([category_filter, severity_filter]))

In [ ]:
# Celda 5: análisis visual de distribución
hist_output = widgets.Output()

# Genera histogramas por categoría y severidad
def update_hist():
    hist_output.clear_output()
    with hist_output:
        if not df_filtered.empty:
            fig1 = px.histogram(df_filtered, x='category', title='Hallazgos por categoría')
            fig2 = px.histogram(df_filtered, x='severity', title='Hallazgos por severidad')
            fig1.show()
            fig2.show()

display(hist_output)

In [ ]:
# Celda 6: exportación a findings.json
export_button = widgets.Button(description='Exportar')
export_output = widgets.Output()

# Guarda el DataFrame filtrado en un archivo JSON validado
def export_data(_):
    df_filtered.to_json('data/findings/findings.json', orient='records', indent=2, force_ascii=False)
    export_output.clear_output()
    with export_output:
        display(Markdown('Exportado a `data/findings/findings.json`'))

export_button.on_click(export_data)

display(export_button, export_output)